In [1]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

from library_dicom.dicom_processor.tools.folders import *

from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *

import numpy as np
import SimpleITK as sitk

In [2]:
#LIST FROM PARSE_FILTERED_DICOM_DATASET

In [2]:
nifti_directory = '/media/salim/DD 2To/GAINED_NIFTI'

csv_directory = '/media/salim/DD 2To/GAINED_CSV_DATA/Export_Gained/CSV41/PET0'

In [3]:
json_path = '/media/salim/DD 2To/GAINED_NIFTI/GAINED_list_path_csv.json'

In [5]:
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

In [6]:
print(dataset[0])

['/media/salim/DD 2To/GAINED_Validated_DICOMS/11011101021001/1.2.840.113704.1.111.3296.1350645932.9/1.3.46.670589.28.2.10.48.18483.298.2.1036.0.1350647998/1.3.46.670589.28.2.10.48.18483.298.2.1036.0.1350647998', 'PT', '/media/salim/DD 2To/GAINED_Validated_DICOMS/11011101021001/1.2.840.113704.1.111.3296.1350645932.9/1.2.840.113704.1.111.5352.1350646167.8/1.2.840.113704.1.111.5352.1350646167.8', 'CT', 'review8.csv', '11011101021001']


In [ ]:
#GENERE LA LISTE DES JSON AVEC UN UNCONSTANT SPACING, SAVE IT AS JSON 

In [7]:
liste_unconstant_z_spacing = []
for liste in dataset : 
    print(dataset.index(liste))
    for i in range(0,4,2) : 
        serie_objet = Series(liste[i])
        serie_objet.get_numpy_array()
        if serie_objet.get_z_spacing() == 'Unconstant Spacing' : 
            print(liste)
            liste_unconstant_z_spacing.append(liste)
            list_z_spacing = serie_objet.calculate_z_spacing() #round = True dans les param
            print(list_z_spacing)
            #liste_unconstant_z_spacing.append(list_z_spacing)


#save it as json

write_json_file('/media/salim/DD 2To/GAINED_NIFTI', 'GAINED_problem_unconstant_spacing', liste_unconstant_z_spacing)

9982, -3.2700000000000387, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.2700000000000387, -3.269999999999982, -3.269999999999982, -3.2700000000000387, -3.269999999999982, -3.269999999999982, -3.2700000000000387, -3.269999999999982, -3.2700000000000387, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.2700000000000955, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.2700000000000955, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.2700000000000955, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.2700000000000955, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.2700000000000955, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.269999999999982, -3.2

In [ ]:
#On enleve les series avec un unconstant spacing , a checker manuellement si gros ecart (manque des slices ?) ou non 
print(len(liste_unconstant_z_spacing)) #
for serie in liste_unconstant_z_spacing : 
    dataset.remove(serie)

print(len(dataset)) #

In [ ]:
#SAVE AS JSON THE FINAL LIST WITHOUT UNCONSTANT SPACING ERROR

In [ ]:
write_json_file('/media/salim/DD 2To/GAINED_NIFTI', 'GAINED_final_list_dataset', dataset)


In [30]:
#FINAL LIST WITHOUT UNCONSTANT SPACING ERROR
json_path = '/media/salim/DD 2To/GAINED_NIFTI/GAINED_final_list_dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

589


In [ ]:
#IF MASK IS CORRECT => GENERATE NIFTI 
#IF SMALL ERRORS EN ELLIPSE = > GENERATE NIFTI 

#IF ERROR ON POLYGONE => MIP 
#CHECK FLIP

In [ ]:
#On met les erreurs de coté

error = []
false_mask = []


for serie in dataset : 
    try : 
        #print(serie)
        print(dataset.index(serie))
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array
            if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                print("MASK CORRECT")
                    #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                    #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
                    #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")
            else : 
                print("FALSE MASK")
                false_mask.append(serie)


        else : 
            serie_pt_objet = SeriesPT(serie[2])
            manufacturer = serie_pt_objet.get_series_details()['series']['Manufacturer'] 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array
            if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                print("MASK CORRECT")
                    #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                     #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
                    #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")
            else : 
                print("FALSE MASK")
                false_mask.append(serie)
               

    except Exception as err : 
        print(err)
        print(serie)
        error.append(serie)

        


In [ ]:
#SAVE THE SERIE WITH FALSE MASK 
write_json_file('/media/salim/DD 2To/GAINED_NIFTI', 'GAINED_false_mask', false_mask)
print(len(false_mask))

In [ ]:
#ON VA TESTER LE FLIP SUR LES FALSE MASK 
#On met les erreurs de coté

error_2 = []
false_mask_after_flip = []


for serie in false_mask : 
    try : 
        #print(serie)
        print(false_mask.index(serie))
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array
            mask_flip = mask_objet.flip_z(nifti_array)

            if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                print("MASK FLIPPE CORRECT")
                    #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                    #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
                    #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_flip)
                print("EXPORT NIFTI MASK")
            else : 
                print("FALSE MASK")
                false_mask_after_flip.append(serie)


        else : 
            serie_pt_objet = SeriesPT(serie[2])
            manufacturer = serie_pt_objet.get_series_details()['series']['Manufacturer'] 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array
            mask_flip = mask_objet.flip_z(nifti_array)

            if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                print("MASK FLIPPE CORRECT")
                    #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                     #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
                    #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_flip)
                print("EXPORT NIFTI MASK")
            else : 
                print("FALSE MASK")
                false_mask_after_flip.append(serie)
               
                
                   


    except Exception as err : 
        print(err)
        print(serie)
        error_2.append(serie)

        
write_json_file('/media/salim/DD 2To/GAINED_NIFTI', 'GAINED_false_mask_after_flip', false_mask_after_flip)
write_json_file('/media/salim/DD 2To/GAINED_NIFTI', 'GAINED_error_2', error_2)

In [ ]:
#GENERER MIP DES MASK FAUX

In [4]:
#SERIE WITH ERROR SUV MAX POLYGONE
json_path = '/media/salim/DD 2To/AHL2011_MIP/AHL2011_false_mask.json'
false_mask = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        false_mask.append(info)

print(len(false_mask))

95


In [7]:
#GENERATE PDF WITH MIP ANGLE 0 OF PET AND MASK FOR EVERY SERIE

from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *
mip_directory = '/media/salim/DD 2To/AHL2011_MIP'



path_mip = []
for serie in false_mask : 
    index = false_mask.index(serie)
    print(index)
    subliste = []
    if serie[1] =='PT' :
        serie_pt_objet = SeriesPT(serie[0])
        study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
        nifti_array = serie_pt_objet.get_numpy_array()
        angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
        subliste.append(angle_filename)

        size_matrix = serie_pt_objet.get_size_matrix()
        mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
        mask_4D = mask_objet.mask_array
        number_roi = mask_4D.shape[3]
        threshold = mask_objet.details_rois['SUVlo']
        #seuillage
        mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array,  flip = False)
        angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, borne_max = 5.0)
        #mip.png MASK
        subliste.append(angle_filename_mask)

        path_mip.append(subliste)


    else : 
        serie_pt_objet = SeriesPT(serie[2])
        study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
        nifti_array = serie_pt_objet.get_numpy_array()
        angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0) 
        #mip.png PET
        subliste.append(angle_filename)
        size_matrix = serie_pt_objet.get_size_matrix()
        mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
        mask_4D = mask_objet.mask_array
        number_roi = mask_4D.shape[3]
        threshold = mask_objet.details_rois['SUVlo']
        #seuillage
        mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array, flip = False)
        angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, borne_max = 5.0)
        #mip.png MASK
        subliste.append(angle_filename_mask)

        path_mip.append(subliste)



0
taille mask :  (128, 128, 313, 10)
1
taille mask :  (128, 128, 275, 16)
2
taille mask :  (128, 128, 275, 5)
3
taille mask :  (128, 128, 275, 30)
4
taille mask :  (128, 128, 275, 20)
5
taille mask :  (128, 128, 287, 46)
6
taille mask :  (128, 128, 287, 15)
7
taille mask :  (168, 168, 405, 70)
8
taille mask :  (168, 168, 479, 38)
9
taille mask :  (200, 200, 481, 21)
10
taille mask :  (168, 168, 390, 6)
11
taille mask :  (200, 200, 481, 19)
12
taille mask :  (200, 200, 481, 5)
13
taille mask :  (192, 192, 299, 12)
14
taille mask :  (256, 256, 335, 13)
15
taille mask :  (256, 256, 299, 13)
16
taille mask :  (256, 256, 299, 27)
17
taille mask :  (256, 256, 299, 8)
18
taille mask :  (256, 256, 335, 16)
19
taille mask :  (256, 256, 335, 10)
20
taille mask :  (192, 192, 299, 8)
21
taille mask :  (256, 256, 299, 20)
22
taille mask :  (256, 256, 299, 18)
23
taille mask :  (256, 256, 335, 13)
24
taille mask :  (256, 256, 299, 24)
25
taille mask :  (256, 256, 299, 11)
26
taille mask :  (256, 256

In [8]:
print(len(path_mip))
write_json_file('/media/salim/DD 2To/AHL2011_MIP', 'AHL2011_list_mip', path_mip)

95


In [9]:
#create pdf 

mip_directory = '/media/salim/DD 2To/AHL2011_MIP'
filename = os.path.join(mip_directory, 'list_mip.pdf')
create_pdf_mip(path_mip, filename)

In [ ]:
#GENERER MIP MASK CORONAL 

In [4]:
# A FAIRE AUSSI LES SERIES EN CORONAL (FLIP Z) 
mip_directory = '/media/salim/DD 2To/AHL2011_MIP_FLIP'



path_mip = []
for serie in list_coronal : 
    index = list_coronal.index(serie)
    print(index)
    subliste = []
    if serie[1] =='PT' :
        serie_pt_objet = SeriesPT(serie[0])
        study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
        nifti_array = serie_pt_objet.get_numpy_array()
        angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
        subliste.append(angle_filename)

        size_matrix = serie_pt_objet.get_size_matrix()
        mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
        mask_4D = mask_objet.mask_array

        print(mask_4D.shape)
        number_roi = mask_4D.shape[3]
        threshold = mask_objet.details_rois['SUVlo']
        #seuillage
        flip_mask = mask_objet.flip_z(mask_4D)
        print(flip_mask.shape)
        #mask_4D_flip = mask_objet.mask_array
        new_mask_4D_flip = threshold_mask(flip_mask, mask_objet.details_rois, nifti_array)
        angle_filename_mask = mip_projection_4D(new_mask_4D_flip, 0, mip_directory, study_uid, number_roi, borne_max = 5.0)
        #mip.png MASK
        subliste.append(angle_filename_mask)

        path_mip.append(subliste)


    else : 
        serie_pt_objet = SeriesPT(serie[2])
        study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
        nifti_array = serie_pt_objet.get_numpy_array()
        angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0) 
        #mip.png PET
        subliste.append(angle_filename)
        size_matrix = serie_pt_objet.get_size_matrix()
        mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
        
        mask_4D = mask_objet.mask_array

        print(mask_4D.shape)
        number_roi = mask_4D.shape[3]
        threshold = mask_objet.details_rois['SUVlo']
        #seuillage

        flip_mask = mask_objet.flip_z(mask_4D)
        
        print(flip_mask.shape)
        #mask_4D_flip = mask_objet.mask_array
        new_mask_4D_flip = threshold_mask(flip_mask, mask_objet.details_rois, nifti_array)
        angle_filename_mask = mip_projection_4D(new_mask_4D_flip, 0, mip_directory, study_uid, number_roi, borne_max = 5.0)
        #mip.png MASK
        subliste.append(angle_filename_mask)

        path_mip.append(subliste)


0
(144, 144, 234, 21)
(144, 144, 234, 21)
taille mask :  (144, 144, 234, 21)
1
(144, 144, 234, 30)
(144, 144, 234, 30)
taille mask :  (144, 144, 234, 30)
2
(144, 144, 213, 13)
(144, 144, 213, 13)
taille mask :  (144, 144, 213, 13)
3
(144, 144, 276, 15)
(144, 144, 276, 15)
taille mask :  (144, 144, 276, 15)
4
(144, 144, 276, 31)
(144, 144, 276, 31)
taille mask :  (144, 144, 276, 31)
5
(144, 144, 276, 15)
(144, 144, 276, 15)
taille mask :  (144, 144, 276, 15)
6
(144, 144, 234, 11)
(144, 144, 234, 11)
taille mask :  (144, 144, 234, 11)
7
(144, 144, 255, 5)
(144, 144, 255, 5)
taille mask :  (144, 144, 255, 5)
8
(256, 256, 335, 25)
(256, 256, 335, 25)
taille mask :  (256, 256, 335, 25)
9
(256, 256, 299, 11)
(256, 256, 299, 11)
taille mask :  (256, 256, 299, 11)
10
(168, 168, 405, 32)
(168, 168, 405, 32)
taille mask :  (168, 168, 405, 32)
11
(256, 256, 515, 12)
(256, 256, 515, 12)
taille mask :  (256, 256, 515, 12)
12
(144, 144, 255, 4)
(144, 144, 255, 4)
taille mask :  (144, 144, 255, 4)
13

In [8]:
mip_directory = '/media/salim/DD 2To/AHL2011_MIP_FLIP'
filename = os.path.join(mip_directory, 'list_mip_coronal.pdf')
create_pdf_mip(path_mip, filename)

In [ ]:
#ALGO GENERATION DE CT PET MASK NIFTI SI MASK VALIDE

In [ ]:
#REGENERER MASK AVEC MOYENNE Z SPACING / ATTENTION AU FLIP !

In [3]:
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_final.json'
csv_directory = ''
nifti_directory = '/media/salim/DD 2To/AHL2011_NIFTI'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

578


In [ ]:
liste_study_uid = []
dataset_coronal = []
for study_uid in liste_study_uid : 
    for data in dataset :
        if study_uid in data[0] : 
            dataset_coronal.append(data)

print(len(dataset_coronal))

In [5]:
for serie in dataset_coronal : 
    print(dataset_coronal.index(serie))
    try : 
        if serie[1] == 'PT' : 
                serie_pt_objet = SeriesPT(serie[0]) 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
                print(study_uid)
                size_matrix = serie_pt_objet.get_size_matrix()
                mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
                mask_4D = mask_objet.mask_array
                flip_mask = mask_objet.flip_z(mask_4D)
 
                print("MASK CORRECT DEJA VERIFIE")
                        #generation nifti PT
                #filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                #serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT DEJA FAIT")
                        #generation nifti CT
                #serie_ct_objet = SeriesCT(serie[2])
                #serie_ct_objet.get_numpy_array()
                #filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                #serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT DEJA FAIT")
                        #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), flip_mask)
                print("EXPORT NIFTI MASK")

        else : 
                serie_pt_objet = SeriesPT(serie[2]) 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
                print(study_uid)
                size_matrix = serie_pt_objet.get_size_matrix()
                mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
                mask_4D = mask_objet.mask_array
                flip_mask = mask_objet.flip_z(mask_4D)

                print("MASK CORRECT DEJA VERIFIE")
                        #generation nifti PT
                #filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                #serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT DEJA FAIT")
                        #generation nifti CT
                #serie_ct_objet = SeriesCT(serie[0])
                #serie_ct_objet.get_numpy_array()
                #filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                #serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT DEJA FAIT")
                        #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), flip_mask)
                print("EXPORT NIFTI MASK")
 

    except Exception as err : 
        print(serie)
        print(err)

Spacing
Unconstant Spacing
EXPORT NIFTI MASK
447
1.2.840.113704.1.111.5300.1325148357.4
MASK CORRECT DEJA VERIFIE
EXPORT NIFTI PT DEJA FAIT
EXPORT NIFTI CT DEJA FAIT
EXPORT NIFTI MASK
448
1.2.250.1.59.470.940100027.1.20130802113038.1925.124326
MASK CORRECT DEJA VERIFIE
EXPORT NIFTI PT DEJA FAIT
EXPORT NIFTI CT DEJA FAIT
EXPORT NIFTI MASK
449
2.16.56.465769650.2551326252.3334845199
MASK CORRECT DEJA VERIFIE
EXPORT NIFTI PT DEJA FAIT
EXPORT NIFTI CT DEJA FAIT
EXPORT NIFTI MASK
450
2.16.840.1.113669.632.20.210791.20000015228
MASK CORRECT DEJA VERIFIE
EXPORT NIFTI PT DEJA FAIT
EXPORT NIFTI CT DEJA FAIT
EXPORT NIFTI MASK
451
1.2.276.0.7230010.3.1.2.304510217.3640.1394037573.43
MASK CORRECT DEJA VERIFIE
EXPORT NIFTI PT DEJA FAIT
EXPORT NIFTI CT DEJA FAIT
EXPORT NIFTI MASK
452
1.2.840.113704.1.111.2128.1346412067.18
MASK CORRECT DEJA VERIFIE
EXPORT NIFTI PT DEJA FAIT
EXPORT NIFTI CT DEJA FAIT
EXPORT NIFTI MASK
453
1.2.840.113619.2.290.3.2831170562.614.1354777093.712
MASK CORRECT DEJA VERIFIE


In [3]:
#VERIFICATION
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_final.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

578


In [6]:
#POUR PET 2 ET PET 4 (SANS MASK)

#RELOAD JSON FROM PARSED DICOM IPYNB


json_path = '/media/salim/DD 2To/AHL2011_NIFTI_PET24/AHL2011_list_path_CT_PET.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

1196


In [5]:
#CHECKER LES UNCONSTANT SPACING
unconstant_spacing = []
serie_error = []
for serie in dataset : 
    print(dataset.index(serie))
    try : 
        if serie[1] =='PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PET")
                unconstant_spacing.append(serie)
            else : 
                serie_ct_objet = SeriesCT(serie[2])
                nifti_array = serie_ct_objet.get_numpy_array()
                if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                    unconstant_spacing.append(serie)

        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing CT")
                unconstant_spacing.append(serie)
            else : 
                serie_ct_objet = SeriesCT(serie[0])
                nifti_array = serie_ct_objet.get_numpy_array()
                if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                    unconstant_spacing.append(serie)

    except Exception as err : 
        print(serie)
        print(err)
        serie_error.append(serie)

write_json_file('/media/salim/DD 2To/AHL2011_NIFTI_PET24', 'AHL2011_serie_error', serie_error)
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI_PET24', 'AHL2011_unconstant_spacing', unconstant_spacing)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
print(len(dataset))
error_1 = dataset[915]
error_2  = dataset[1117]

dataset.remove(error_1)
dataset.remove(error_2)
print(len(dataset))

1196
1194


In [9]:
#GENERATION NIFTI CT PET DES PET 2/4

nifti_directory = '/media/salim/DD 2To/AHL2011_NIFTI_PET24'
for serie in dataset : 
    print(dataset.index(serie))
    try : 
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")
                    #generation nifti CT
            serie_ct_objet = SeriesCT(serie[2])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")



        else : 
            serie_pt_objet = SeriesPT(serie[2]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

            
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")
                    #generation nifti CT
            serie_ct_objet = SeriesCT(serie[0])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")
 

    except Exception as err : 
        print(serie)
        print(err)

EXPORT NIFTI PT
EXPORT NIFTI CT
685
EXPORT NIFTI PT
EXPORT NIFTI CT
686
EXPORT NIFTI PT
EXPORT NIFTI CT
687
EXPORT NIFTI PT
EXPORT NIFTI CT
688
EXPORT NIFTI PT
EXPORT NIFTI CT
689
EXPORT NIFTI PT
EXPORT NIFTI CT
690
EXPORT NIFTI PT
EXPORT NIFTI CT
691
EXPORT NIFTI PT
EXPORT NIFTI CT
692
EXPORT NIFTI PT
EXPORT NIFTI CT
693
EXPORT NIFTI PT
EXPORT NIFTI CT
694
EXPORT NIFTI PT
EXPORT NIFTI CT
695
EXPORT NIFTI PT
EXPORT NIFTI CT
696
EXPORT NIFTI PT
EXPORT NIFTI CT
697
EXPORT NIFTI PT
EXPORT NIFTI CT
698
EXPORT NIFTI PT
EXPORT NIFTI CT
699
EXPORT NIFTI PT
EXPORT NIFTI CT
700
EXPORT NIFTI PT
EXPORT NIFTI CT
701
EXPORT NIFTI PT
EXPORT NIFTI CT
702
EXPORT NIFTI PT
EXPORT NIFTI CT
703
EXPORT NIFTI PT
EXPORT NIFTI CT
704
EXPORT NIFTI PT
EXPORT NIFTI CT
705
EXPORT NIFTI PT
EXPORT NIFTI CT
706
EXPORT NIFTI PT
EXPORT NIFTI CT
707
EXPORT NIFTI PT
EXPORT NIFTI CT
708
EXPORT NIFTI PT
EXPORT NIFTI CT
709
EXPORT NIFTI PT
EXPORT NIFTI CT
710
EXPORT NIFTI PT
EXPORT NIFTI CT
711
EXPORT NIFTI PT
EXPORT NIFTI